In [ ]:
import requests
import time
import json
from dotenv import load_dotenv
import os

# Load the environment variables from secrets.env
load_dotenv('secrets.env')

class GraphAPIClient:
    def __init__(self, tenant_id, client_id, client_secret, scope, access_token=None):
        self.tenant_id = tenant_id
        self.client_id = client_id
        self.client_secret = client_secret
        self.scope = scope
        self.access_token = access_token
        self.base_token_url = f'https://login.microsoftonline.com/{self.tenant_id}/oauth2/v2.0/token'

    def authenticate(self):
        request_body = {
            'client_id': self.client_id,
            'scope': self.scope,
            'client_secret': self.client_secret,
            'grant_type': 'client_credentials'
        }
        response = requests.post(self.base_token_url, data=request_body)
        if response.status_code == 200:
            self.access_token = response.json()['access_token']
        else:
            raise Exception(f"Authentication failed: {response.status_code}, {response.content}")

    def post_billing_request(self, api_url, billing_period, currency_code):
        if not self.access_token:
            raise Exception("Access token is missing. Authenticate first.")
        
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        body = {
            'billingPeriod': billing_period,
            'currencyCode': currency_code
        }

        response = requests.post(api_url, headers=headers, json=body)

        if response.status_code == 202:
            print('Request accepted. Processing has started.')
            return response.headers  # Return headers to use the URL for checking status
        else:
            raise Exception(f"Failed to make request. Status code: {response.status_code}. Content: {response.content}")

    def check_operation_status(self, operation_url):
        if not self.access_token:
            raise Exception("Access token is missing. Authenticate first.")

        headers = {
            'Authorization': f'Bearer {self.access_token}'
        }

        while True:
            response = requests.get(operation_url, headers=headers)
            
            if response.status_code in [401, 403]:
                raise Exception("Access token is expired or invalid. Please refresh the token and try again.")
            elif response.status_code == 200:
                json_data = response.json()
                status = json_data.get('status')
                print(f"Status: {status}")

                if status in ['succeeded', 'failed']:
                    return json_data  # Return final result

                retry_after = response.headers.get('Retry-After', 10)
                print(f"Retrying after {retry_after} seconds...")
                time.sleep(int(retry_after))
            else:
                raise Exception(f"Request failed. Status code: {response.status_code}. Response: {response.content}")

# Example usage
if __name__ == "__main__":
    # Load credentials from the .env file
    tenant_id = os.getenv('TENANT_ID')
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    scope = os.getenv('SCOPE')

    # Initialize the client and authenticate
    client = GraphAPIClient(tenant_id, client_id, client_secret, scope)
    try:
        client.authenticate()
        print("Authenticated successfully.")

        # Make the billing request
        billing_api_url = 'https://graph.microsoft.com/v1.0/reports/partners/billing/usage/unbilled/export'
        billing_period = 'current'
        currency_code = 'INR'
        headers = client.post_billing_request(billing_api_url, billing_period, currency_code)
        
        # Fetch the operation URL from headers (assuming it's in 'Location')
        operation_url = headers.get('Location')
        if operation_url:
            result = client.check_operation_status(operation_url)
            print(json.dumps(result, indent=2))
        else:
            print("Operation URL not found in the response headers.")
    
    except Exception as e:
        print(f"Error: {e}")


In [6]:
class Animal():
    num_of_teeth = 32

    def __init__(self, sound, food):
        self.animals_sound = sound
        self.animals_food = food
        self.animals_type = 'Generic Animal'
    
    def make_sound(self):
        print(f"{self.animals_type} makes a sound : {self.animals_sound} ")

    def eat_fav_food(self):
        print(f"{self.animals_type} eats its favorite food : {self.animals_food}")

    def get_num_of_teeth(self):
        return self.num_of_teeth



In [7]:
# cat = Animal('meow', 'mouse')
cat = Animal('meow', 'mouse')
dog = Animal('woof', 'bone')


In [21]:
class Cow(Animal):
    num_of_teeth = 22

    def __init__(self, name, breed):
        super().__init__('moo', 'grass')
        self.name = name
        self.breed = breed
        self.animals_type = self.name

In [22]:
cow = Cow('basanti','breed1')

In [ ]:
cow.make_sound()

In [5]:
class Car:
    def __init__(self, colour, engine_size, model, weight, top_speed):
        self.colour = colour
        self.engine_size = engine_size
        self.model = model
        self.weight = weight
        self.top_speed = top_speed
        self.current_speed = 0

    def start(self):
        print(f"{self.model} has started")

    def increase_speed(self, speed_increase):
        current_speed = self.current_speed
        self.current_speed += speed_increase
        print(f"Current speed was {current_speed}, and now has increased  to {self.current_speed} mph")  

    def stop(self):
        print(f"{self.model} has stopped")

    
              
    

    

In [6]:
my_car = Car('red', 4500, 'swift', 1000, 180)

In [ ]:
my_car.increase_speed(20)


In [ ]:
my_car.current_speed

In [ ]:
import requests
import pandas as pd
from secret_manager import SecretsManager
from datetime import datetime, timedelta

# Initialize SecretsManager to fetch credentials
secrets = SecretsManager()

class PartnerCenterAPIClient:
    def __init__(self, base_url: str, client_id: str, client_secret: str, tenant_id: str, 
                 invoice_url: str, invoice_line_items_url: str, scope: str):
        """
        Initializes the PartnerCenterAPIClient with the provided credentials and URLs.

        Args:
            base_url (str): Base URL for the Partner Center API.
            client_id (str): Client ID for OAuth authentication.
            client_secret (str): Client secret for OAuth authentication.
            tenant_id (str): Tenant ID for OAuth authentication.
            invoice_url (str): URL for fetching invoices.
            invoice_line_items_url (str): URL for fetching invoice line items.
            scope (str): Scope for OAuth authentication.
        """
        self.base_url = base_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.tenant_id = tenant_id
        self.scope = scope  
        self.invoice_url = invoice_url
        self.invoice_line_items_url = invoice_line_items_url

    def get_access_token(self) -> str:
        """
        Obtains an access token for authentication with the Partner Center API.

        Returns:
            str: Access token for authorization.
        """
        token_url = f"https://login.microsoftonline.com/{self.tenant_id}/oauth2/v2.0/token"
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        body = {
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'scope': self.scope, 
            'grant_type': 'client_credentials'
        }

        response = requests.post(token_url, data=body, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
        else:
            raise Exception(f"Failed to authenticate: {response.status_code}, {response.content}")

        return self.access_token

    def get_invoice_ids(self) -> list[dict]:
        """
        Retrieves the list of invoice IDs from the Partner Center API.

        Returns:
            list[dict]: List of invoice data with IDs and billing dates.
        """
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        response = requests.get(f"{self.invoice_url}", headers=headers)
        if response.status_code == 200:
            return response.json().get('items', [])
        else:
            raise Exception(f"Error fetching invoices: {response.status_code}, {response.content}")

    def filter_invoices(self, invoice_ids: list[dict]) -> list[str]:
        """
        Filters invoice IDs to find those corresponding to the previous month.

        Args:
            invoice_ids (list[dict]): List of invoice data to filter.

        Returns:
            list[str]: List of filtered invoice IDs for the previous month.
        """
        # Filter invoices to find those starting with 'G'
        filtered_invoices = [invoice for invoice in invoice_ids if invoice['id'].startswith('G')]

        # Initialize max_date to None
        max_date = None

        # Parse billingPeriodStartDate to datetime and find the maximum billing period date
        for invoice in filtered_invoices:
            billing_date = datetime.strptime(invoice['billingPeriodStartDate'].replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
            if max_date is None or billing_date > max_date:
                max_date = billing_date

        # Get the current month - 1
        last_month = (datetime.now().replace(day=1) - pd.DateOffset(months=1)).strftime('%Y-%m')

        # Compare max_date with last_month
        if max_date:
            max_month_str = max_date.strftime('%Y-%m')
            if max_month_str == last_month:
                # Collect IDs for the invoices matching last_month
                matching_invoice_ids = [invoice['id'] for invoice in filtered_invoices if invoice['billingPeriodStartDate'].startswith(last_month)]
                return matching_invoice_ids

        return []

    def get_invoice_line_items(self, invoice_id: str) -> list[dict]:
        """
        Retrieves line items for a specific invoice ID from the Partner Center API.

        Args:
            invoice_id (str): The ID of the invoice to fetch line items for.

        Returns:
            list[dict]: List of line items associated with the invoice.
        """
        # Replace the invoice ID in the line items URL
        line_items_url = self.invoice_line_items_url.replace("<invoiceID>", invoice_id)

        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }

        response = requests.get(line_items_url, headers=headers)
        if response.status_code == 200:
            return response.json().get('items', [])
        else:
            raise Exception(f"Error fetching invoice line items for {invoice_id}: {response.status_code}, {response.content}")



def main():
    """
    Main function to execute the workflow of fetching invoices, processing data,
    and interacting with the datawarehouse.
    """
    # Load credentials from secrets
    base_url = secrets.partner_api_base_url
    client_id = secrets.client_id
    client_secret = secrets.client_secret
    tenant_id = secrets.tenant_id
    invoice_url = secrets.invoice_url
    invoice_line_items_url = secrets.invoice_line_item_url
    scope = secrets.scope
    
    # Initialize API client
    api_client = PartnerCenterAPIClient(base_url, client_id, client_secret, tenant_id, invoice_url, invoice_line_items_url, scope)

    try:
        print("Fetching access token...")
        access_token = api_client.get_access_token()
        print("Access token retrieved successfully.")

        # Fetch invoice IDs
        print("Fetching invoice IDs...")
        invoice_ids = api_client.get_invoice_ids()
        print(f"Retrieved {len(invoice_ids)} invoice IDs that starts with G.")

        # Filter invoices and find matching months
        print("Filtering invoices to find last month invoice...")
        matching_invoice_ids = api_client.filter_invoices(invoice_ids)

        if matching_invoice_ids:
            print(f"Found matching invoice ID(s): {matching_invoice_ids}")
            invoice_id = matching_invoice_ids[0]
            print(f"Fetching line items for invoice ID: {invoice_id}...")
            line_items = api_client.get_invoice_line_items(invoice_id)

            # Add billing_month information
            current_month_minus_1 = (datetime.now().replace(day=1) - timedelta(days=1)).strftime('%Y-%m-01')
            for item in line_items:
                item['billing_month'] = current_month_minus_1

            # Print the line items for the selected invoice ID
            print(f"Line items for invoice ID {invoice_id} for billing month {current_month_minus_1}:")
            for item in line_items:
                # Remove the "attributes" key from the item if it exists
                item.pop("attributes", None)  # Remove the key, if it exists
                print(item)  # Printing each line item

        else:
            print("No matching invoices for the previous month.")

    except Exception as e:
        print(f"An error occurred: {e}")
        # You could add more specific error handling here based on the context of the error


if __name__ == "__main__":
    main()

In [ ]:
import requests
from secret_manager import SecretsManager
from datetime import datetime
import os
import json
from collections import defaultdict


# Initialize SecretsManager to fetch credentials
secrets = SecretsManager()

class PartnerCenterAPIClient:
    def __init__(self, base_url: str, client_id: str, client_secret: str, tenant_id: str, 
                 invoice_url: str, invoice_line_items_url: str, scope: str):
        """
        Initializes the PartnerCenterAPIClient with the provided credentials and URLs.
        """
        self.base_url = base_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.tenant_id = tenant_id
        self.scope = scope  
        self.invoice_url = invoice_url
        self.invoice_line_items_url = invoice_line_items_url

    def get_access_token(self) -> str:
        """
        Obtains an access token for authentication with the Partner Center API.
        """
        token_url = f"https://login.microsoftonline.com/{self.tenant_id}/oauth2/v2.0/token"
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        body = {
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'scope': self.scope, 
            'grant_type': 'client_credentials'
        }

        response = requests.post(token_url, data=body, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
        else:
            raise Exception(f"Failed to authenticate: {response.status_code}, {response.content}")

        return self.access_token

    def get_invoice_ids(self) -> list[dict]:
        """
        Retrieves the list of invoice IDs from the Partner Center API.
        """
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        response = requests.get(f"{self.invoice_url}", headers=headers)
        if response.status_code == 200:
            return response.json().get('items', [])
        else:
            raise Exception(f"Error fetching invoices: {response.status_code}, {response.content}")

    def filter_invoices_for_july_august(self, invoice_ids: list[dict]) -> list[str]:
        """
        Filters invoice IDs to find those for July and August that start with 'G'.

        Args:
            invoice_ids (list[dict]): List of invoice data to filter.

        Returns:
            list[str]: List of filtered invoice IDs for July and August that start with 'G'.
        """
        # Define the months to filter for (July and August)
        july = '2023-07'
        august = '2023-08'

        # Filter invoices based on billing period start date and ID starting with 'G'
        filtered_invoices = [
            invoice for invoice in invoice_ids 
            if (invoice['billingPeriodStartDate'].startswith(july) or 
                invoice['billingPeriodStartDate'].startswith(august)) and 
            invoice['id'].startswith('G')
        ]
        
        # Extract invoice IDs
        matching_invoice_ids = [invoice['id'] for invoice in filtered_invoices]

        return matching_invoice_ids


    def download_invoice_line_items(self, invoice_id: str, download_dir: str = "./downloads") -> dict:
        """
        Downloads the line items for a specific invoice ID, adds billing_month,
        checks subscriptionStartDate and subscriptionEndDate, and saves them as CSV files.

        Args:
            invoice_id (str): The ID of the invoice to download.
            download_dir (str): The directory to save the downloaded files.

        Returns:
            dict: A dictionary with the paths of the downloaded CSV files by month.
        """
        # Replace the invoice ID in the line items URL
        line_items_url = self.invoice_line_items_url.replace("<invoiceID>", invoice_id)

        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }

        response = requests.get(line_items_url, headers=headers)
        if response.status_code == 200:
            line_items = response.json().get('items', [])

            # Dictionary to store line items by billing month
            items_by_month = defaultdict(list)

            # Process each item to add billing_month and handle subscriptionStartDate and subscriptionEndDate
            for item in line_items:
                # Remove the "attributes" field if it exists
                if 'attributes' in item:
                    del item['attributes']
                
                # Add or compute billing_month based on chargeStartDate
                charge_start_date = item.get('chargeStartDate', None)
                if charge_start_date:
                    try:
                        # Clean and format the charge_start_date
                        charge_start_date_cleaned = charge_start_date.rstrip('Z')
                        if '.' in charge_start_date_cleaned:
                            charge_start_date_cleaned = charge_start_date_cleaned.split('.')[0] + '.' + charge_start_date_cleaned.split('.')[1][:6]

                        billing_month = datetime.strptime(charge_start_date_cleaned, "%Y-%m-%dT%H:%M:%S.%f").strftime('%Y-%m-01')
                    except ValueError:
                        charge_start_date_cleaned = charge_start_date.rstrip('Z')
                        billing_month = datetime.strptime(charge_start_date_cleaned, "%Y-%m-%dT%H:%M:%S").strftime('%Y-%m-01')

                    item['billing_month'] = billing_month

                    # Add the item to the respective month in the dictionary
                    items_by_month[billing_month].append(item)
                
                # Handle subscriptionStartDate and subscriptionEndDate
                item['subscriptionStartDate'] = item.get('subscriptionStartDate', '')
                item['subscriptionEndDate'] = item.get('subscriptionEndDate', '')

                # Remove the "productQualifiers" field if it exists
                if 'productQualifiers' in item:
                    del item['productQualifiers']

            # Ensure the download directory exists
            os.makedirs(download_dir, exist_ok=True)

            # Dictionary to store paths of the downloaded files
            file_paths = {}

            # Now we write separate CSV files for each month
            for billing_month, items in items_by_month.items():
                # Extract month and year for the file name
                billing_date = datetime.strptime(billing_month, '%Y-%m-01')
                month_name = billing_date.strftime('%B')  # Get full month name like "August"
                year = billing_date.strftime('%Y')  # Get year like "2023"

                # Define the CSV file path
                file_name = f"{month_name}_{year}.csv"  # E.g., "August_2023.csv"
                file_path = os.path.join(download_dir, file_name)

                # Get the keys (column headers) for the CSV
                if items:
                    headers = list(items[0].keys())
                else:
                    headers = []  # If there are no items, no headers

                # Save the line items as a CSV file
                with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
                    writer = csv.DictWriter(csv_file, fieldnames=headers)
                    writer.writeheader()  # Write the header row
                    writer.writerows(items)  # Write the data rows

                print(f"Invoice line items for {invoice_id} saved successfully as CSV to {file_path}")
                file_paths[billing_month] = file_path

            return file_paths
        else:
            raise Exception(f"Error fetching invoice line items for {invoice_id}: {response.status_code}, {response.content}")




def main():
    """
    Main function to fetch and download invoices for July and August.
    """
    # Load credentials from secrets
    base_url = secrets.partner_api_base_url
    client_id = secrets.client_id
    client_secret = secrets.client_secret
    tenant_id = secrets.tenant_id
    invoice_url = secrets.invoice_url
    invoice_line_items_url = secrets.invoice_line_item_url
    scope = secrets.scope
    
    # Initialize API client
    api_client = PartnerCenterAPIClient(base_url, client_id, client_secret, tenant_id, invoice_url, invoice_line_items_url, scope)

    try:
        print("Fetching access token...")
        access_token = api_client.get_access_token()
        print("Access token retrieved successfully.")

        # Fetch invoice IDs
        print("Fetching invoice IDs...")
        invoice_ids = api_client.get_invoice_ids()
        print(f"Retrieved {len(invoice_ids)} invoice IDs.")

        # Filter invoices for July and August that start with 'G'
        print("Filtering invoices for July and August that start with 'G'...")
        matching_invoice_ids = api_client.filter_invoices_for_july_august(invoice_ids)

        if matching_invoice_ids:
            print(f"Found matching invoice ID(s): {matching_invoice_ids}")

            # Download each matching invoice's line items
            for invoice_id in matching_invoice_ids:
                print(f"Downloading line items for invoice ID: {invoice_id}...")
                api_client.download_invoice_line_items(invoice_id)

        else:
            print("No matching invoices for July or August that start with 'G'.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import csv
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from secret_manager import SecretsManager
from datetime import datetime, timedelta
import requests

# Initialize SecretsManager to fetch credentials
secrets = SecretsManager()

class PartnerCenterAPIClient:
    def __init__(self, base_url: str, client_id: str, client_secret: str, tenant_id: str, 
                 invoice_url: str, invoice_line_items_url: str, scope: str, blob_connection_string: str, blob_container_name: str):
        """
        Initializes the PartnerCenterAPIClient with the provided credentials and URLs.

        Args:
            base_url (str): Base URL for the Partner Center API.
            client_id (str): Client ID for OAuth authentication.
            client_secret (str): Client secret for OAuth authentication.
            tenant_id (str): Tenant ID for OAuth authentication.
            invoice_url (str): URL for fetching invoices.
            invoice_line_items_url (str): URL for fetching invoice line items.
            scope (str): Scope for OAuth authentication.
            blob_connection_string (str): Azure Blob Storage connection string.
            blob_container_name (str): Name of the container in Azure Blob Storage.
        """
        self.base_url = base_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.tenant_id = tenant_id
        self.scope = scope  
        self.invoice_url = invoice_url
        self.invoice_line_items_url = invoice_line_items_url
        self.blob_connection_string = blob_connection_string
        self.blob_container_name = blob_container_name

        # Initialize BlobServiceClient
        self.blob_service_client = BlobServiceClient.from_connection_string(self.blob_connection_string)

    def get_access_token(self) -> str:
        """
        Obtains an access token for authentication with the Partner Center API.

        Returns:
            str: Access token for authorization.
        """
        token_url = f"https://login.microsoftonline.com/{self.tenant_id}/oauth2/v2.0/token"
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        body = {
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'scope': self.scope, 
            'grant_type': 'client_credentials'
        }

        response = requests.post(token_url, data=body, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
        else:
            raise Exception(f"Failed to authenticate: {response.status_code}, {response.content}")

        return self.access_token

    def get_invoice_ids(self) -> list[dict]:
        """
        Retrieves the list of invoice IDs from the Partner Center API.

        Returns:
            list[dict]: List of invoice data with IDs and billing dates.
        """
        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
        response = requests.get(f"{self.invoice_url}", headers=headers)
        if response.status_code == 200:
            return response.json().get('items', [])
        else:
            raise Exception(f"Error fetching invoices: {response.status_code}, {response.content}")

    def filter_invoices(self, invoice_ids: list[dict]) -> list[str]:
        """
        Filters invoice IDs to find those corresponding to July 2024.

        Args:
            invoice_ids (list[dict]): List of invoice data to filter.

        Returns:
            list[str]: List of filtered invoice IDs for July 2024.
        """
        # Filter invoices to find those starting with 'G'
        filtered_invoices = [invoice for invoice in invoice_ids if invoice['id'].startswith('G')]

        # Set the target month to July 2024
        target_month = '2024-07'

        # Collect IDs for the invoices matching the target month
        matching_invoice_ids = [invoice['id'] for invoice in filtered_invoices if invoice['billingPeriodStartDate'].startswith(target_month)]

        return matching_invoice_ids

    def get_invoice_line_items(self, invoice_id: str) -> list[dict]:
        """
        Retrieves line items for a specific invoice ID from the Partner Center API.

        Args:
            invoice_id (str): The ID of the invoice to fetch line items for.

        Returns:
            list[dict]: List of line items associated with the invoice.
        """
        # Replace the invoice ID in the line items URL
        line_items_url = self.invoice_line_items_url.replace("<invoiceID>", invoice_id)

        headers = {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }

        response = requests.get(line_items_url, headers=headers)
        if response.status_code == 200:
            return response.json().get('items', [])
        else:
            raise Exception(f"Error fetching invoice line items for {invoice_id}: {response.status_code}, {response.content}")

    def write_to_blob_storage(self, line_items: list[dict], invoice_id: str):
        """
        Writes the line items to Azure Blob Storage in CSV format.

        Args:
            line_items (list[dict]): The line items to write.
            invoice_id (str): The ID of the invoice, used for naming the file.
        """
        # Create a CSV in memory
        csv_output = StringIO()
        writer = csv.DictWriter(csv_output, fieldnames=line_items[0].keys())
        writer.writeheader()
        writer.writerows(line_items)

        # Get blob client for the container and blob (using invoice_id for filename)
        blob_client = self.blob_service_client.get_blob_client(container=self.blob_container_name, blob=f"invoice_{invoice_id}.csv")

        # Upload CSV content to Azure Blob Storage
        blob_client.upload_blob(csv_output.getvalue(), overwrite=True)
        print(f"Successfully uploaded invoice {invoice_id} line items to Azure Blob Storage.")

def main():
    """
    Main function to execute the workflow of fetching invoices, processing data,
    and writing the line items to Azure Blob Storage.
    """
    # Load credentials from secrets
    base_url = secrets.partner_api_base_url
    client_id = secrets.client_id
    client_secret = secrets.client_secret
    tenant_id = secrets.tenant_id
    invoice_url = secrets.invoice_url
    invoice_line_items_url = secrets.invoice_line_item_url
    scope = secrets.scope
    blob_connection_string = secrets.blob_connection_string
    blob_container_name = secrets.blob_container_name

    # Initialize API client
    api_client = PartnerCenterAPIClient(base_url, client_id, client_secret, tenant_id, invoice_url, invoice_line_items_url, scope, blob_connection_string, blob_container_name)

    try:
        print("Fetching access token...")
        access_token = api_client.get_access_token()
        print("Access token retrieved successfully.")

        # Fetch invoice IDs
        print("Fetching invoice IDs...")
        invoice_ids = api_client.get_invoice_ids()
        print(f"Retrieved {len(invoice_ids)} invoice IDs that starts with G.")

        # Filter invoices and find matching months
        print("Filtering invoices to find July 2024 invoice...")
        matching_invoice_ids = api_client.filter_invoices(invoice_ids)

        if matching_invoice_ids:
            print(f"Found matching invoice ID(s): {matching_invoice_ids}")
            invoice_id = matching_invoice_ids[0]
            print(f"Fetching line items for invoice ID: {invoice_id}...")
            line_items = api_client.get_invoice_line_items(invoice_id)

            # Add billing_month information
            current_month_minus_1 = '2024-07-01'
            for item in line_items:
                item['billing_month'] = current_month_minus_1

            # Write line items to Blob Storage
            api_client.write_to_blob_storage(line_items, invoice_id)

        else:
            print("No matching invoices for July 2024.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()
